In [2]:
import pandas as pd
import os
root = '../data/寶典/'
files = os.listdir(root)
dfs = []
for f in files:
    try:
        path = root+f
        xls = pd.ExcelFile(path)
        sheets = xls.sheet_names
        df = pd.read_excel(path,sheet_name=sheets[-1])[['CODIV','DIVNM','ITEMNM']].dropna(axis=0)
        dfs.append(df)
        print(f,df.shape)
    except:
        pass


寶典.v3.台塑網.20210901.xlsx (1099, 3)
寶典.v4.20211001.xlsx (1100, 3)
寶典.v5.20211006.xlsx (1103, 3)
寶典.v6.20211020.xlsx (1109, 3)


In [3]:
a = dfs[-1]
a[a['ITEMNM']=='GASOLINE']

,CODIV,DIVNM,ITEMNM
1047,60,國外處,GASOLINE
1057,61,台塑石化,GASOLINE
1108,6B,油品事業部,GASOLINE


In [4]:
import xlrd

# Load xls file
data = xlrd.open_workbook('..\data\寶典\寶典.v6.20211020.xlsx')
for n in range(len(data.sheet_names())):
    print(n)

0


In [1]:
import pandas as pd
from pandas.io.excel._openpyxl import _OpenpyxlReader
from pandas._typing import Scalar
from typing import List
from pandas.io.excel._odfreader import _ODFReader
from pandas.io.excel._xlrd import _XlrdReader

class CustomReader(_OpenpyxlReader):
    def get_sheet_data(self, sheet, convert_float: bool) -> List[List[Scalar]]:
        data = []
        for row in sheet.rows:
            first = row[1] # I need the strikethrough check on this cell only
            if first.value is not None and first.font.strike: continue
            else:
                data.append([self._convert_cell(cell, convert_float) for cell in row])
        return data

class CustomExcelFile(pd.ExcelFile):

    _engines = {"xlrd": _XlrdReader, "openpyxl": CustomReader, "odf": _ODFReader}

In [3]:
excel = CustomExcelFile('..\data\寶典\寶典.v6.20211020.xlsx', engine="openpyxl")
df = excel.parse()
print(df.shape)
df[df.ITEMNM=='GASOLINE']

(1104, 6)


,RIGID,EXTN,CODIV,DIVNM,ITEMNM,memo
1044,NaN,NaN,61,NaN,GASOLINE,10/20+
1049,陳惠慈,7237,61,台塑石化,GASOLINE,NaN


In [6]:
df.groupby('ITEMNM')['CODIV'].nunique().sort_values(ascending=False)

ITEMNM
PVC COPOLYMER                3
ALKYLATE                     3
PP RESIN                     3
KNITTED FABRIC               2
GASOLINE                     2
                            ..
POLYETHYLENE PE              1
POLYETHYLENE LLDPE3470       1
 TAIRILIN BRAND PET CHIPS    1
PROPYLENE COPOLYMER          0
BUTANDIOL                    0
Name: CODIV, Length: 1046, dtype: int64

In [7]:
df[df.ITEMNM=='PVC COPOLYMER']

,RIGID,EXTN,CODIV,DIVNM,ITEMNM,memo
6,黃淑玲,7190,11,台塑塑膠部,PVC COPOLYMER,NaN
230,李佩珍,6445,18,台塑聚烯部,PVC COPOLYMER,NaN
787,NaN,NaN,2P,南亞塑四部化學品部,PVC COPOLYMER,NaN
